In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('FrozenLake-v0')
env.observation_space.n

[2019-03-18 14:23:33,833] Making new env: FrozenLake-v0
/home/b/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


16

In [3]:
env.action_space.n

4

In [4]:
value_table = np.zeros(env.observation_space.n)
n_iter = 100000

In [5]:
threshold = 1e-20

In [6]:
def value_iter(env, gamma=1.0):
    for i in range(n_iter):
        updated_value_table = np.copy(value_table)

        for state in range(env.observation_space.n):
            Q_value = []

            for action in range( env.action_space.n):
                next_states_rewards = []

                for next_sr in env.env.P[state][action]:
                    trans_prob, next_state, reward_prob, _ = next_sr
                    next_states_rewards.append((trans_prob*(reward_prob+gamma*updated_value_table[next_state])))
                Q_value.append(np.sum(next_states_rewards))
            value_table[state]=max(Q_value)
        if np.sum(np.fabs(updated_value_table-value_table))<=threshold:
            print('converged at iter{}'.format(i))
            break
    return value_table

In [22]:
%%time
optimal_value_function = value_iter(env=env, gamma=1.0)

converged at iter0
CPU times: user 1e+03 µs, sys: 339 µs, total: 1.34 ms
Wall time: 1.29 ms


In [8]:
env.close()

In [9]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [10]:
optimal_value_function

array([0.82352941, 0.82352941, 0.82352941, 0.82352941, 0.82352941,
       0.        , 0.52941176, 0.        , 0.82352941, 0.82352941,
       0.76470588, 0.        , 0.        , 0.88235294, 0.94117647,
       0.        ])

In [11]:
def extract_policy(value_table, gamma = 1.0, env=env.env):
 
    # initialize the policy with zeros
    policy = np.zeros(env.observation_space.n) 
    
    
    for state in range(env.observation_space.n):
        
        # initialize the Q table for a state
        Q_table = np.zeros(env.action_space.n)
        
        # compute Q value for all ations in the state
        for action in range(env.action_space.n):
            for next_sr in env.P[state][action]: 
                trans_prob, next_state, reward_prob, _ = next_sr 
                Q_table[action] += (trans_prob * (reward_prob + gamma * value_table[next_state]))
        
        # select the action which has maximum Q value as an optimal action of the state
        policy[state] = np.argmax(Q_table)
    
    return policy

In [12]:
extract_policy(optimal_value_function)

array([0., 3., 3., 3., 0., 0., 0., 0., 3., 1., 0., 0., 0., 2., 1., 0.])

In [19]:
def compute_value_function(policy, gamma=1.0):
    
    # initialize value table with zeros
    value_table = np.zeros(env.env.nS)
    
    # set the threshold
    threshold = 1e-10
    
    while True:
        
        # copy the value table to the updated_value_table
        updated_value_table = np.copy(value_table)

        # for each state in the env.environment, select the action according to the policy and compute the value table
        for state in range(env.env.nS):
            action = policy[state]
            
            # build the value table with the selected action
            value_table[state] = sum([trans_prob * (reward_prob + gamma * updated_value_table[next_state]) 
                        for trans_prob, next_state, reward_prob, _ in env.env.P[state][action]])
            
        if (np.sum((np.fabs(updated_value_table - value_table))) <= threshold):
            break
            
    return value_table

In [17]:
def policy_iteration(env,gamma = 1.0):
    
    # Initialize policy with zeros
    old_policy = np.zeros(env.observation_space.n)   
    no_of_iterations = 200000
    
    for i in range(no_of_iterations):
        
        # compute the value function
        new_value_function = compute_value_function(old_policy, gamma)
        
        # Extract new policy from the computed value function
        new_policy = extract_policy(new_value_function, gamma)
   
        # Then we check whether we have reached convergence i.e whether we found the optimal
        # policy by comparing old_policy and new policy if it same we will break the iteration
        # else we update old_policy with new_policy

        if (np.all(old_policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        old_policy = new_policy
        
    return new_policy

In [21]:
%%time
policy_iteration(env)

Policy-Iteration converged at step 7.
CPU times: user 112 ms, sys: 17.3 ms, total: 129 ms
Wall time: 119 ms


array([0., 3., 3., 3., 0., 0., 0., 0., 3., 1., 0., 0., 0., 2., 1., 0.])